In [ ]:
from os.path import join

from os_paw.wfs_api import WFS_API
import configparser
import geopandas as gpd
from pygeos.io import from_shapely
import pickle as pkl

import project_paths as paths

In [ ]:
config = configparser.ConfigParser()
config.read(paths.config_path)
API_KEY = config['KEYS']['API_KEY']

# Choose a Spatial Reference System
SRS = 'EPSG:27700'

# Choose an OS Web Feature Service product
TYPE_NAME = 'Highways_RoadLink'

# Create Bounding Box
CENTRE_COORD = (441826, 112231)
MARGIN = 500
bbox = f"{CENTRE_COORD[0] - MARGIN}, {CENTRE_COORD[1] - MARGIN}, {CENTRE_COORD[0] + MARGIN}, {CENTRE_COORD[1] + MARGIN}"
print(bbox)

In [ ]:
# Create WFS_API object and run query
wfs_api = WFS_API(api_key=API_KEY)
payload = wfs_api.get_all_features_within_bbox(type_name=TYPE_NAME,
                                               bbox=BBOX,
                                               srs=SRS,
                                               allow_premium=True)

In [ ]:
roads = gpd.GeoDataFrame.from_features(payload["features"], crs=SRS)

In [ ]:
roads.plot()

In [ ]:
roads.to_file("../tests/data/sample_roads_soton.geojson", driver="GeoJSON")

In [ ]:
roads = gpd.read_file(r"..\tests\data\sample_roads_soton_centre.geojson", driver="GeoJSON")

In [ ]:
roads = roads.explode().reset_index()
geometries = list(roads.geometry)
toids = list(roads["ID"])
sample_roads = {toid: list(geometry.coords) for toid, geometry in zip(toids, geometries)}
with open("../tests/data/sample_roads.pickle", 'wb') as file_name:
    pkl.dump(sample_roads, file_name)


In [ ]:
sample_roads